In [1]:
from math import log
import operator

def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],
               [0, 0, 0, 1, 'no'],
               [0, 1, 0, 1, 'yes'],
               [0, 1, 1, 0, 'yes'],
               [0, 0, 0, 0, 'no'],
               [1, 0, 0, 0, 'no'],
               [1, 0, 0, 1, 'no'],
               [1, 1, 1, 1, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [2, 0, 1, 2, 'yes'],
               [2, 0, 1, 1, 'yes'],
               [2, 1, 0, 1, 'yes'],
               [2, 1, 0, 2, 'yes'],
               [2, 0, 0, 0, 'no']]
    labels = ['F1-AGE', 'F2-WORK', 'F3-HOME', 'F4-LOAN']
    return dataSet, labels

# 计算数据集的香农熵
def calcShannonEnt(dataset):
    # 获取数据集的样本数量
    numEntries = len(dataset)
    # 用于存储每个类别出现的次数
    labelCounts = {}
    # 遍历数据集中的每个样本
    for featVec in dataset:
        # 获取当前样本的类别标签
        currentLabel = featVec[-1]
        # 如果该类别标签还未在 labelCounts 中
        if currentLabel not in labelCounts.keys():
            # 初始化该类别标签的计数为 0
            labelCounts[currentLabel] = 0
        # 该类别标签的计数加 1
        labelCounts[currentLabel] += 1
    # 初始化香农熵为 0
    shannonEnt = 0.0
    # 遍历每个类别标签及其计数
    for key in labelCounts:
        # 计算该类别出现的概率
        prob = float(labelCounts[key]) / numEntries
        # 根据香农熵公式更新香农熵
        shannonEnt -= prob * log(prob, 2)
    # 返回计算得到的香农熵
    return shannonEnt

# 根据指定特征和特征值划分数据集
def splitDataSet(dataset, axis, value):
    retDataSet = []
    for featVec in dataset:
        if featVec[axis] == value:
            # 创建一个不包含指定划分特征的新样本
            # 提取当前样本中指定特征之前的部分
            reducedFeatVec = featVec[:axis]
            # 将当前样本中指定特征之后的部分添加到 reducedFeatVec 中
            reducedFeatVec.extend(featVec[axis + 1:])    
            # 将处理后的样本添加到划分后的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet

# 选择最佳的特征进行数据集划分
def chooseBestFeatureToSplit(dataset):
    # 获取特征的数量，减去 1 是因为最后一列是类别标签
    numFeatures = len(dataset[0]) - 1
    # 计算数据集的初始香农熵
    baseEntropy = calcShannonEnt(dataset)
    # 初始化最大信息增益为 0
    bestInfoGain = 0.0
    # 初始化最佳特征的索引为 -1
    bestFeature = -1
    # 遍历每个特征
    for i in range(numFeatures):
        # 获取该特征在所有样本中的值
        featList = [example[i] for example in dataset]
        # 获取该特征所有不同的值
        uniqueVals = set(featList)
        # 初始化划分后的香农熵为 0
        newEntropy = 0.0
        # 遍历该特征的每个不同值
        for value in uniqueVals:
            # 根据该特征的当前值划分数据集
            subDataSet = splitDataSet(dataset, i, value)
            # 计算该子集在数据集中所占的比例
            prob = len(subDataSet) / float(len(dataset))
            # 累加该子集的香农熵到 newEntropy
            newEntropy += prob * calcShannonEnt(subDataSet)
        # 计算信息增益
        infoGain = baseEntropy - newEntropy
        # 如果当前信息增益大于最大信息增益
        if infoGain > bestInfoGain:
            # 更新最大信息增益
            bestInfoGain = infoGain
            # 更新最佳特征的索引
            bestFeature = i
    # 返回最佳特征的索引
    return bestFeature

# 当无法继续划分时，返回出现次数最多的类别
def majorityCnt(classList):
    # 用于存储每个类别出现的次数
    classCount = {}
    # 遍历类别列表
    for vote in classList:
        # 如果该类别还未在 classCount 中
        if vote not in classCount.keys():
            # 初始化该类别的计数为 0
            classCount[vote] = 0
        # 该类别的计数加 1
        classCount[vote] += 1
    # 对类别计数进行排序，按照计数从大到小排列
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse=True)
    # 返回出现次数最多的类别
    return sortedClassCount[0][0]

# 递归地构建决策树
def createTree(dataset, labels, featLabels):
    # 获取数据集中所有样本的类别标签
    classList = [example[-1] for example in dataset]
    # 如果所有样本的类别标签都相同
    if classList.count(classList[0]) == len(classList):
        # 直接返回该类别标签
        return classList[0]
    # 如果数据集中只剩下类别标签这一列
    if len(dataset[0]) == 1:
        # 返回出现次数最多的类别标签
        return majorityCnt(classList)
    # 选择最佳的特征进行划分
    bestFeat = chooseBestFeatureToSplit(dataset)
    # 获取最佳特征的名称
    bestFeatLabel = labels[bestFeat]
    # 将最佳特征的名称添加到特征标签列表中
    featLabels.append(bestFeatLabel)
    # 初始化决策树，以最佳特征为根节点
    myTree = {bestFeatLabel: {}}
    # 从标签列表中删除最佳特征的名称
    del (labels[bestFeat])
    # 获取最佳特征在所有样本中的值
    featValues = [example[bestFeat] for example in dataset]
    # 获取最佳特征所有不同的值
    uniqueVals = set(featValues)
    # 遍历最佳特征的每个不同值
    for value in uniqueVals:
        # 复制标签列表，避免修改原始列表
        subLabels = labels[:]
        # 递归地构建子树
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataset, bestFeat, value), subLabels, featLabels)
    return myTree

if __name__ == "__main__":
    # 创建示例数据集和特征标签列表
    dataset, labels = createDataSet()
    featLabels = []
    myTree = createTree(dataset, labels, featLabels)
    print(myTree)           

{'F3-HOME': {0: {'F2-WORK': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
